<a href="https://colab.research.google.com/github/yamayama3689/finance/blob/main/%E6%A0%AA%E5%BC%8F%E6%83%85%E5%A0%B1%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd
import csv

In [ ]:
# colabのシークレットに保存しトークン情報を取得
from google.colab import userdata
J_QUANTS_EMAIL = userdata.get('J_QUANTS_EMAIL')
J_QUANTS_PASSWORD = userdata.get('J_QUANTS_PASSWORD')

In [ ]:
# 上場銘柄一覧取得
# リフレッシュトークン取得

# 登録したメールアドレス、パスワードを設定
# EMAIL_ADDRESSに登録メールアドレス、PASSWORDにパスワードを入力
data={"mailaddress":J_QUANTS_EMAIL, "password":J_QUANTS_PASSWORD}

# リフレッシュトークン取得
res_ref_token = requests.post("https://api.jquants.com/v1/token/auth_user", data=json.dumps(data)) # data(Pythonの辞書形式)をjson形式に変えてやる

In [ ]:
# IDトークンを取得

# リフレッシュトークンを設定
REFRESH_TOKEN = res_ref_token.json()['refreshToken']

# IDトークンを取得
res_id_token = requests.post(f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}")

In [ ]:
# 全銘柄の取得
# freeプランでは、12週間より前のデータしか取得できないことに注意

# 取得したIDトークンを設定（取得から24時間まで有効）
id_token = res_id_token.json()['idToken']
headers = {'Authorization': 'Bearer {}'.format(id_token)}
DATE = '2025-05-01' # 取得可能な期間のうち、最新の月（ただし月初）の期間を指定する

# 銘柄リスト一覧を取得
res_list = requests.get(f"https://api.jquants.com/v1/listed/info?date={DATE}", headers=headers)
res_list.json()

In [ ]:
# pandasのデータフレームに変換
df_code_list = pd.DataFrame(res_list.json()['info'])

In [ ]:
# データの確認
df_code_list.head()
# Date:取得日付
# Code:銘柄コード
# CompanyName:会社名
# Sector17Code:17業種コード
# Sector33Code:33業種コード
# ScaleCategory:規模コード
# MarketCode:市場区分コード その他にはETFが含まれている

,Date,Code,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName
0,2025-05-01,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
1,2025-05-01,13050,大和アセットマネジメント株式会社 ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,iFreeETF TOPIX (Yearly Dividend Type),99,その他,9999,その他,-,0109,その他
2,2025-05-01,13060,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,NEXT FUNDS TOPIX Exchange Traded Fund,99,その他,9999,その他,-,0109,その他
3,2025-05-01,13080,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,Listed Index Fund TOPIX,99,その他,9999,その他,-,0109,その他
4,2025-05-01,13090,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０...,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,99,その他,9999,その他,-,0109,その他


In [ ]:
df_code_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4415 entries, 0 to 4414
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                4415 non-null   object
 1   Code                4415 non-null   object
 2   CompanyName         4415 non-null   object
 3   CompanyNameEnglish  4415 non-null   object
 4   Sector17Code        4415 non-null   object
 5   Sector17CodeName    4415 non-null   object
 6   Sector33Code        4415 non-null   object
 7   Sector33CodeName    4415 non-null   object
 8   ScaleCategory       4415 non-null   object
 9   MarketCode          4415 non-null   object
 10  MarketCodeName      4415 non-null   object
dtypes: object(11)
memory usage: 379.5+ KB


In [ ]:
# 会社名の全角スペース（\u3000）を半角スペースに置き換え
df_code_list['CompanyName'] = df_code_list['CompanyName'].str.replace('\u3000',' ')

# オブジェクト型から文字列型に変更
df_code_list = df_code_list.astype('string')

In [ ]:
# 個別銘柄のみを抽出する
# 「TOKYO PRO MARKET(0105)」「その他(0109)」の市場を除外
df_code_list_individual = df_code_list[(df_code_list['MarketCode'] != '0105') & (df_code_list['MarketCode'] != '0109')]

df_code_list_individual.reset_index(inplace=True, drop=True)

In [ ]:
df_code_list_individual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3831 entries, 0 to 3830
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                3831 non-null   string
 1   Code                3831 non-null   string
 2   CompanyName         3831 non-null   string
 3   CompanyNameEnglish  3831 non-null   string
 4   Sector17Code        3831 non-null   string
 5   Sector17CodeName    3831 non-null   string
 6   Sector33Code        3831 non-null   string
 7   Sector33CodeName    3831 non-null   string
 8   ScaleCategory       3831 non-null   string
 9   MarketCode          3831 non-null   string
 10  MarketCodeName      3831 non-null   string
dtypes: string(11)
memory usage: 329.4 KB


In [ ]:
df_code_list_individual.head()

,Date,Code,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName
0,2025-05-01,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
1,2025-05-01,130A0,Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,Veritas In Silico Inc.,5,医薬品,3250,医薬品,-,0113,グロース
2,2025-05-01,13320,ニッスイ,Nissui Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム
3,2025-05-01,13330,マルハニチロ,Maruha Nichiro Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム
4,2025-05-01,135A0,ＶＲＡＩＮ Ｓｏｌｕｔｉｏｎ,"VRAIN Solution,Inc.",10,情報通信・サービスその他,5250,情報･通信業,-,0113,グロース


In [ ]:
import os

# 現在の作業ディレクトリを取得
current_directory = os.getcwd()

print(current_directory)

# 目的のディレクトリ
new_directory = '/content/drive/MyDrive/investment'

# ディレクトリを変更
os.chdir(new_directory)

# 変更後のディレクトリを確認
print(os.getcwd())

/content/drive/MyDrive/investment
/content/drive/MyDrive/investment


In [ ]:
# CSVファイルに保存
df_code_list_individual.to_csv('sample_code_list.csv', index=False, encoding='utf-8')

In [ ]:
import yfinance as yf
import pandas as pd
from google.cloud import storage
import datetime
import io

# 銘柄と期間を指定
ticker_symbol = "AAPL"
start_date = "2024-01-01"
end_date = datetime.date.today().strftime("%Y-%m-%d")

# yfinanceで株価データを取得
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# データをメモリ上でCSV形式に変換
csv_data = data.to_csv(index=False)
csv_bytes = csv_data.encode('utf-8')

# Google Cloud Storageに直接アップロード
bucket_name = "your-gcs-bucket-name"
file_name = f"stock_data/{ticker_symbol}_{end_date}.csv"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

blob.upload_from_file(io.BytesIO(csv_bytes), content_type='text/csv')

print(f"Successfully uploaded {file_name} to {bucket_name}.")